In [1]:
import itertools
import math
from scipy.optimize import least_squares

In [2]:
def compute_shapley(w):
    """
    Compute Shapley values for each role given penalties.
    
    Parameters:
    - w: list or tuple [w_CM, w_B, w_CI]
    
    Returns:
    - dict with Shapley values for 'AI','B','CI','CM'
    """
    w_CM, w_B, w_CI = w
    players = ["AI", "B", "CI", "CM"]
    n = len(players)
    
    def v(coalition):
        S = set(coalition)
        # Require AI to be present
        if "AI" not in S:
            return 0.0
        val = 1.0
        # Subtract penalty for missing roles
        if "CM" not in S:
            val -= w_CM
        if "B" not in S:
            val -= w_B
        if "CI" not in S:
            val -= w_CI
        return max(val, 0.0)
    
    shapley = {p: 0.0 for p in players}
    fact = math.factorial
    
    for p in players:
        others = [q for q in players if q != p]
        for k in range(len(others) + 1):
            for subset in itertools.combinations(others, k):
                S = set(subset)
                marginal = v(S.union({p})) - v(S)
                weight = fact(len(S)) * fact(n - len(S) - 1) / fact(n)
                shapley[p] += weight * marginal
    
    return shapley

def find_penalties(target, initial_guess=(0.3, 0.3, 0.1)):
    """
    Find missing-role penalties that match desired Shapley values.
    
    Parameters:
    - target: dict with target Shapley values for 'AI','B','CI','CM' (must sum to 1)
    - initial_guess: starting guess for [w_CM, w_B, w_CI]
    
    Returns:
    - result.x: optimized [w_CM, w_B, w_CI]
    - result: full OptimizeResult from scipy.optimize
    """
    def residuals(w):
        sh = compute_shapley(w)
        # match B, CI, and CM; AI will automatically sum to ~1
        return [
            sh["B"]  - target["B"],
            sh["CI"] - target["CI"],
            sh["CM"] - target["CM"]
        ]
    
    # Bounds: each penalty between 0 and 1
    result = least_squares(residuals, initial_guess, bounds=(0,1))
    return result.x, result

In [3]:
# Example usage:
# Desired split: AI 60%, B 25%, CI 5%, CM 10%
target_shapley = {"AI": 0.60, "B": 0.25, "CI": 0.05, "CM": 0.10}
penalties, sol = find_penalties(target_shapley)

print("Optimized penalties:")
print(f"  w_CM = {penalties[0]:.4f}")
print(f"  w_B  = {penalties[1]:.4f}")
print(f"  w_CI = {penalties[2]:.4f}")

final_shap = compute_shapley(penalties)
print("\nResulting Shapley values:")
for role, val in final_shap.items():
    print(f"  {role}: {val:.4f}")


Optimized penalties:
  w_CM = 0.2000
  w_B  = 0.5000
  w_CI = 0.1000

Resulting Shapley values:
  AI: 0.6000
  B: 0.2500
  CI: 0.0500
  CM: 0.1000
